In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import re


In [2]:
# List of unpleasant words to check
unpleasant = ['lousy','disappointed','discouraged','ashamed','powerless','diminished','guilty','dissatisfied',
        'miserable','detestable','repugnant','despicable','disgusting','abominable','terrible','in despair',
        'sulky','bad','upset','doubtful','uncertain','indecisive','perplexed','embarassed','hesitant',
        'shy','stupefied','disillusioned','unbelieving','skeptical','distrustful','misgiving','lost',
        'unsure','uneasy','pessimistic','tense''incapable','alone','paralyzed','fatigued','useless',
        'inferior','vulnerable','empty','forced','hesitant','despair','frustrated','distressed','woeful',
        'pathetic','tragic','in a stew','dominated','irritated','enraged','hostile','insulting',
        'annoyed','upset','hateful','offensive','bitter','aggresive','resentful','inflamed','provoked',
        'incensed','infuriated','cross','worked up','boiling','fuming','fearful','terrified','suspicious',
        'anxious','alarmed','panic','nervous','scared','worried','frightened','timid','shaky','restless',
        'doubtful','threatened','cowardly','quaking','wary','crushed','tormented','deprived','pained',
        'tortured','dejected','rejected','injured','offended','afflicted','aching','victimized',
        'heartbroken','agonized','appalled','humiliated','wronged','alienated','tearful','sorrowful',
        'pained','grief','anguish','desolate','desperate','pessimistic','unhappy','lonely','grieved',
        'mournful','dismayed','insensitive','dull','nonchalant','neutral','reserved','weary','bored',
        'preoccupied','cold','disinterested','lifeless', 'never'];

In [3]:
# List of pleasant words to check
pleasant = ['understanding','confident','reliable','easy','amazed','free','sympathetic','interested','satisfied',
            'receptive','accepting','kind', 'great','joyous','lucky','fortunate','delighted','overjoyed','gleeful','thankful','important',
            'festive','ecstatic','glad','cheerful','elated','jubilant', 'playful','courageous','energetic','liberated','optimistic',
            'impulsive','free','animated','spirited','thrilled','wonderful','calm','peaceful','at ease','comfortable','pleased','encouraged',
            'clever','surprised','content','quiet','certain','relaxed','serene','reassured', 'loving','considerate','affectionate', 'sensitive',
            'tender','devoted','attracted','passionate','admiration','warm','touched','close','comforted','loved','concerned','affected','fascinated',
            'intrigued','absorbed','inquisitive','engrossed','curious','drawn toward','eager','keen','earnest','intent','inspired','determined','excited',
            'enthusiastic','bold','brave','daring',
            'optimistic','impulsive','free','sure','certain','rebellious','unique','dynamic','tenacious','hardy','secure',
            'confident','challenged', 'love'];

In [4]:
# Read csv
path = 'Resources/Amazon_rev.csv'

In [5]:
amazon_foods_df = pd.read_csv(path)

In [6]:
amazon_foods_df.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
# Drop first column because it creates junk info
amazon_foods_df = amazon_foods_df.drop(columns=["Unnamed: 0"])

In [8]:
amazon_foods_modified_df = amazon_foods_df

In [9]:
# Add a regular expression to look for all the symbols
regex = "[.\"\',!@#$%^&*()\_\-+=?:;|/!]"

In [10]:
# Tex replacement for all rules in the regex
text_replace = ''

In [11]:
#Create first column with text in lower case and no special characters
amazon_foods_modified_df['Lowercase Text'] = ''

In [12]:
# Split data frame to run demo version
amazon_foods_modified_df = amazon_foods_df.iloc[:50]

In [13]:
# Pupulate new column column 
amazon_foods_modified_df['Lowercase Text'] = amazon_foods_df.apply(
    lambda row: (
            re.sub(
                regex,
                text_replace,
                row[9].lower()
            )
    ),
    axis=1)

/Users/Gugunner-Oryx-Pro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [14]:
#Create second column and clear br html tags
amazon_foods_modified_df['Lowercase Text Clean'] = ''

/Users/Gugunner-Oryx-Pro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
# Define function to extract with beautifull soup br and just return text property
def extract_br_tags(soup):
    for e in soup.findAll('br'):
        e.extract()
    if soup.find('p'):
        return soup.find('p').text
    return soup

In [16]:
# Define function to count positive and negative words based on pleasant and unpleasant lists
def add_positive_negative(text, is_positive):
    text_list = text.split()
#     print(text_list)
    count = 0
    for word in text_list:
        if is_positive:
            for pleasant_word in pleasant:
                if word == pleasant_word:
                    count+=1
        else:
            for unpleasant_word in unpleasant:
                if word == unpleasant_word:
                    count+=1
    print(f'Word count {count}')
    return count
    


In [17]:
amazon_foods_modified_df['Lowercase Text Clean'] = amazon_foods_modified_df.apply(
    lambda row: (
        extract_br_tags(bs(row[10],'lxml'))
    ),
    axis=1)

/Users/Gugunner-Oryx-Pro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [18]:
# Create column for positive count
amazon_foods_modified_df['Positive Count'] = ''

/Users/Gugunner-Oryx-Pro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
amazon_foods_modified_df['Positive Count'] = amazon_foods_modified_df.apply(
    lambda row: (
        add_positive_negative(row[11], True)
    ),
    axis=1)

Word count 0
Word count 1
Word count 0
Word count 0
Word count 2
Word count 0
Word count 2
Word count 0
Word count 1
Word count 0
Word count 5
Word count 0
Word count 0
Word count 1
Word count 0
Word count 0
Word count 1
Word count 1
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 1
Word count 0
Word count 0
Word count 0
Word count 2
Word count 5
Word count 5
Word count 1
Word count 2
Word count 1
Word count 1
Word count 1
Word count 2
Word count 1
Word count 0
Word count 0
Word count 0
Word count 1
Word count 2
Word count 2
Word count 0
Word count 0
Word count 0
Word count 1
Word count 0
Word count 4
Word count 1


/Users/Gugunner-Oryx-Pro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
# Create column for negative count
amazon_foods_modified_df['Negative Count'] = ''

/Users/Gugunner-Oryx-Pro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
amazon_foods_modified_df['Negative Count'] = amazon_foods_modified_df.apply(
    lambda row: (
        add_positive_negative(row[11], False)
    ),
    axis=1)

Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 1
Word count 0
Word count 0
Word count 0
Word count 1
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 1
Word count 0
Word count 0
Word count 0
Word count 1
Word count 0
Word count 1
Word count 1
Word count 1
Word count 0
Word count 1
Word count 1
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 2
Word count 3
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 0
Word count 2


/Users/Gugunner-Oryx-Pro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [22]:
amazon_foods_modified_df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Lowercase Text,Lowercase Text Clean,Positive Count,Negative Count
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,i have bought several of the vitality canned d...,i have bought several of the vitality canned d...,0,0
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled as jumbo salted peanut...,product arrived labeled as jumbo salted peanut...,1,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,this is a confection that has been around a fe...,this is a confection that has been around a fe...,0,0
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,if you are looking for the secret ingredient i...,if you are looking for the secret ingredient i...,0,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffy at a great price there was a wide...,great taffy at a great price there was a wide...,2,0
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,i got a wild hair for taffy and ordered this f...,i got a wild hair for taffy and ordered this f...,0,0
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,positive,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,this saltwater taffy had great flavors and was...,this saltwater taffy had great flavors and was...,2,0
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,positive,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,this taffy is so good it is very soft and che...,this taffy is so good it is very soft and che...,0,0
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,positive,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...,right now im mostly just sprouting this so my ...,right now im mostly just sprouting this so my ...,1,0
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,positive,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...,this is a very healthy dog food good for their...,this is a very healthy dog food good for their...,0,0


In [23]:
# Drop columns not needed for modified csv
amazon_foods_modified_df = amazon_foods_modified_df.drop(columns=['Lowercase Text'])

In [24]:
amazon_foods_modified_df.to_csv('Amazon_rev_modified.csv')